
# Documentação do Notebook: Limpeza de Dados de Parlamentares e Emendas Parlamentares

## Objetivo
Este notebook tem como objetivo realizar a limpeza e padronização das tabelas de parlamentares e emendas parlamentares.

## Etapas do Processo
1. **Exclusão de Colunas Irrelevantes**
    - Remoção de colunas que não são necessárias para a análise.

2. **Análise de Valores Faltantes**
    - Identificação e tratamento de valores nulos ou ausentes nas tabelas.

3. **Análise de Valores Inválidos**
    - Verificação e correção de dados inconsistentes ou fora do padrão esperado.

4. **Padronização de Case Sensitive**
    - Uniformização de textos para evitar discrepâncias causadas por diferenças de maiúsculas/minúsculas.

## Tabelas Utilizadas
  - **Parlamentares**
  - **Emendas Parlamentares**

## Resultado Esperado
Ao final do notebook, as tabelas estarão limpas, padronizadas e prontas para análises posteriores.

# Importação de bibliotecas e funções

In [0]:
from pyspark.sql import functions as F

# Tratamentos

## Tabela de emendas parlamentares

In [0]:
# Carregar a tabela de emendas parlamentares
emendas = spark.read.table("mvp.bronze.emendas")


In [0]:
# Visualizar as primeiras 5 linhas
emendas.limit(5).display()

Ano,Tipo_de_Emenda,Autor_da_emenda,Numero_da_emenda,Possui_Apoiador_Solicitante_,Localidade_do_gasto__Regionalizacao_,Funcao,Subfuncao,Programa_Orcamentario,Acao_Orcamentaria,Plano_Orcamentario,Codigo_da_emenda,Valor_empenhado,Valor_liquidado,Valor_pago,Valor_Restos_a_Pagar_Inscritos,Valor_Restos_a_Pagar_Cancelados,Valor_Restos_a_Pagar_Pagos,_c18
2023,Emenda Individual - Transferências com Finalidade Definida,3767 - TADEU ALENCAR,0004,Não se aplica,BODOCÓ - PE,Saúde,Assistência hospitalar e ambulatorial,5018 - ATENCAO ESPECIALIZADA A SAUDE,8535 - ESTRUTURACAO DE UNIDADES DE ATENCAO ESPECIALIZADA EM SAUDE,ESTRUTURACAO DE UNIDADES DE ATENCAO ESPECIALIZADA EM SAUDE - DESPESAS DIVERSAS,202337670004,"6.467,00","6.467,00","6.467,00","0,00","0,00","0,00",null
2023,Emenda Individual - Transferências com Finalidade Definida,4028 - JUNIOR MANO,0006,Não se aplica,TURURU - CE,Saúde,Atenção básica,5019 - ATENCAO PRIMARIA A SAUDE,8581 - ESTRUTURACAO DA REDE DE SERVICOS DE ATENCAO PRIMARIA A SAUDE E SAUDE BUCAL,ESTRUTURACAO DA REDE DE SERVICOS DE ATENCAO PRIMARIA A SAUDE - DESPESAS DIVERSAS,202340280006,"24.570,00","24.570,00","24.570,00","0,00","0,00","0,00",null
2023,Emenda de Comissão,6003 - COM. CONST. JUSTICA E CIDADANIA,0004,Não,Nacional,Direitos da cidadania,"Direitos individuais, coletivos e difusos",5015 - JUSTICA,2334 - PROTECAO E DEFESA DO CONSUMIDOR,PROTECAO E DEFESA DO CONSUMIDOR - DESPESAS DIVERSAS,202360030004,"25.000,00","0,00","0,00","0,00","0,00","25.000,00",null
2023,Emenda de Comissão,5013 - COM. DEFESA DO CONSUMIDOR,0003,Não,Nacional,Direitos da cidadania,"Direitos individuais, coletivos e difusos",5015 - JUSTICA,2334 - PROTECAO E DEFESA DO CONSUMIDOR,PROTECAO E DEFESA DO CONSUMIDOR - DESPESAS DIVERSAS,202350130003,"25.000,00","0,00","0,00","25.000,00","0,00","0,00",null
2023,Emenda Individual - Transferências com Finalidade Definida,2885 - HUMBERTO COSTA,0013,Não se aplica,PERNAMBUCO (UF),Educação,Ensino profissional,5012 - EDUCACAO PROFISSIONAL E TECNOLOGICA,"20RL - FUNCIONAMENTO DAS INSTITUICOES DA REDE FEDERAL DE EDUCACAO PROFISSIONAL, CIENTIFICA E TECNOLOGICA","FUNCIONAMENTO DAS INSTITUICOES DA REDE FEDERAL DE EDUCACAO PROFISSIONAL, CIENTIFICA E TECNOLOGICA",202328850013,"38.997,44","9.995,44","9.871,44","28.451,10","0,00","674,90",null


In [0]:
# Colunas da tabela
emendas.columns

['Ano',
 'Tipo_de_Emenda',
 'Autor_da_emenda',
 'Numero_da_emenda',
 'Possui_Apoiador_Solicitante_',
 'Localidade_do_gasto__Regionalizacao_',
 'Funcao',
 'Subfuncao',
 'Programa_Orcamentario',
 'Acao_Orcamentaria',
 'Plano_Orcamentario',
 'Codigo_da_emenda',
 'Valor_empenhado',
 'Valor_liquidado',
 'Valor_pago',
 'Valor_Restos_a_Pagar_Inscritos',
 'Valor_Restos_a_Pagar_Cancelados',
 'Valor_Restos_a_Pagar_Pagos',
 '_c18']

In [0]:
# Selecionar apenas as colunas desejadas
colunas_relevantes_emendas = ['Ano',
 'Autor_da_emenda',
 'Localidade_do_gasto__Regionalizacao_',
 'Funcao',
 'Valor_empenhado',
 'Valor_liquidado',
 'Valor_pago',
]
emendas_filtradas = emendas.select(colunas_relevantes_emendas)

emendas_filtradas.limit(5).display()

Ano,Autor_da_emenda,Localidade_do_gasto__Regionalizacao_,Funcao,Valor_empenhado,Valor_liquidado,Valor_pago
2023,3767 - TADEU ALENCAR,BODOCÓ - PE,Saúde,"6.467,00","6.467,00","6.467,00"
2023,4028 - JUNIOR MANO,TURURU - CE,Saúde,"24.570,00","24.570,00","24.570,00"
2023,6003 - COM. CONST. JUSTICA E CIDADANIA,Nacional,Direitos da cidadania,"25.000,00","0,00","0,00"
2023,5013 - COM. DEFESA DO CONSUMIDOR,Nacional,Direitos da cidadania,"25.000,00","0,00","0,00"
2023,2885 - HUMBERTO COSTA,PERNAMBUCO (UF),Educação,"38.997,44","9.995,44","9.871,44"


### Análise da colunas

#### Ano

In [0]:
emendas_filtradas.select("Ano").printSchema()

root
 |-- Ano: string (nullable = true)



A coluna de ano está como string e precisará ser alterada para inteiro

In [0]:
emendas_filtradas = emendas_filtradas.withColumn("Ano", F.col("Ano").cast("int"))
emendas_filtradas.select("Ano").printSchema()



root
 |-- Ano: integer (nullable = true)



In [0]:
emendas_filtradas.filter(F.col("Ano").isNull()).count()

0

Não temos valores nulos para a coluna de Ano

In [0]:
emendas_filtradas.groupBy("Ano").count().orderBy(F.desc("count")).display()

Ano,count
2024,6990
2023,6110
2025,5639


Não temos vazios temporais para os anos considerados. O número de emendas de 2025 é menor, mas é próximo ao número de emendas de 2023 e ainda estamos no ano corrente de 2025, possibilitando ainda novas emendas.

#### Autor da Emenda

In [0]:
emendas_filtradas.select("Autor_da_emenda").printSchema()

root
 |-- Autor_da_emenda: string (nullable = true)



O Formato está correto para essa coluna

In [0]:
emendas_filtradas.filter(F.col("Autor_da_emenda").isNull()).count()

0

Não temos valores nulos para autores de emendas

In [0]:
emendas_filtradas.limit(5).display()

Ano,Autor_da_emenda,Localidade_do_gasto__Regionalizacao_,Funcao,Valor_empenhado,Valor_liquidado,Valor_pago
2023,3767 - TADEU ALENCAR,BODOCÓ - PE,Saúde,"6.467,00","6.467,00","6.467,00"
2023,4028 - JUNIOR MANO,TURURU - CE,Saúde,"24.570,00","24.570,00","24.570,00"
2023,6003 - COM. CONST. JUSTICA E CIDADANIA,Nacional,Direitos da cidadania,"25.000,00","0,00","0,00"
2023,5013 - COM. DEFESA DO CONSUMIDOR,Nacional,Direitos da cidadania,"25.000,00","0,00","0,00"
2023,2885 - HUMBERTO COSTA,PERNAMBUCO (UF),Educação,"38.997,44","9.995,44","9.871,44"


Foi notado que temos autores de emendas que não são parlamentares, mas comissões. Vamos investigar mais afundo

In [0]:
emendas_filtradas.groupBy("Autor_da_emenda").count().orderBy(F.desc("count")).limit(5).display()

Autor_da_emenda,count
9219 - FLAVIO BOLSONARO,80
2855 - PEDRO UCZAI,69
1775 - JANDIRA FEGHALI,68
3984 - FERNANDA MELCHIONNA,68
3717 - MARCIO ALVINO,67


Vão ser eliminadas da base de autores:
- Bancadas (BANCADA)
- Comissões (COM.)

Além disso, foi notado que o número de parlamentares contidos na base de emendas está bem superior ao valor obtido na base de parlamentares e vamos precisar investigar melhor qual será o tratamento.


In [0]:
emendas_filtradas = emendas_filtradas.filter(
    ~(
        F.col("Autor_da_emenda").like("%BANCADA%") |
        F.col("Autor_da_emenda").like("%COM.%")
    )
)


In [0]:
emendas_filtradas.groupBy("Autor_da_emenda").count().orderBy(F.asc("count")).limit(10).display()

Autor_da_emenda,count
4135 - VINICIUS POIT,1
"2429 - TIAGO DIMAS (EX-PARLAMENTAR LAZARO BOTELHO, NOS TERMOS ART. 78 LDO 2025 E DA MENSAGEM 95-CN, DE 06.11.25)",1
2670 - JHONATAN DE JESUS,2
3934 - CELINA LEAO,2
2152 - BETO FARO,2
4117 - MARCELO NILO,2
9089 - BOZZELLA,2
3933 - CASSIO ANDRADE,2
4228 - ELIANE NOGUEIRA,2
3080 - DANIEL COELHO,3


Ao buscar nomes como Vinicius Poit, Beto Faro e Marcelo Nilo, observamos que são deputados federais do mandato anterior, cujas emendas foram executadas no ano seguinte, já na nova legislatura. Para nossas análises, esses parlamentares serão desconsiderados, pois não constam na base atual de parlamentares.

In [0]:
emendas_filtradas = emendas_filtradas.withColumn(
    "nome_parlamentar",
    F.upper(
        F.trim(
            F.regexp_extract(
                F.col("Autor_da_emenda"),
                r"-\s*(.*?)(?=\(|$)",
                1
            )
        )
    )
)

A coluna "nome_parlamentar" foi criada extraindo o nome após o caractere "-" até o final do texto, ignorando qualquer conteúdo entre parênteses. O resultado foi convertido para maiúsculas e espaços em branco foram removidos.

In [0]:
emendas_filtradas.groupBy("nome_parlamentar").count().orderBy(F.desc("count")).limit(5).display()

nome_parlamentar,count
FLAVIO BOLSONARO,80
PEDRO UCZAI,69
JANDIRA FEGHALI,68
FERNANDA MELCHIONNA,68
MARCIO ALVINO,67


In [0]:
emendas_filtradas = emendas_filtradas.drop("Autor_da_emenda")

#### Localidade

In [0]:
emendas_filtradas.select("Localidade_do_gasto__Regionalizacao_").printSchema()

root
 |-- Localidade_do_gasto__Regionalizacao_: string (nullable = true)



A Coluna está como string e é o tipo correto para a coluna

In [0]:
emendas_filtradas.filter(F.col("Localidade_do_gasto__Regionalizacao_").isNull()).count()

0

Não temos valores nulos para localidade da emenda parlamentar

In [0]:
emendas_filtradas.groupBy("Localidade_do_gasto__Regionalizacao_").count().orderBy(F.desc("count")).limit(50).display()

Localidade_do_gasto__Regionalizacao_,count
MÚLTIPLO,7120
Nacional,1658
SÃO PAULO (UF),783
RIO DE JANEIRO (UF),699
MINAS GERAIS (UF),562
RIO GRANDE DO SUL (UF),348
BAHIA (UF),343
PERNAMBUCO (UF),341
PARANÁ (UF),298
DISTRITO FEDERAL (UF),292


Ao analisar os valores, foram percebidos três tipos de informações:
- Emendas associadas a cidades
- Emendas associados a estados
- Emendas associadas a regiões, nacionais ou múltiplas

Por isso será realizado o seguinte tratamento:

O tratamento da coluna de localidade padroniza todo o texto para uppercase e remove espaços extras. Quando a localidade está no formato “MUNICÍPIO - UF”, o município é extraído e a UF final é identificada diretamente. Caso contrário, o código tenta identificar uma UF no final da string; se não existir, extrai o nome do estado e utiliza um dicionário de mapeamento para obter a sigla correspondente. A coluna uf_destino segue essa ordem de prioridade: primeiro a UF do padrão de município, depois a UF no final da string e, por fim, a UF obtida via mapeamento do estado.

In [0]:
# Dicionário com o mapeamento de estados para siglas
uf_map = {
    "ACRE": "AC","ALAGOAS": "AL","AMAPÁ": "AP","AMAPA": "AP","AMAZONAS": "AM",
    "BAHIA": "BA","CEARÁ": "CE","CEARA": "CE","DISTRITO FEDERAL": "DF",
    "ESPÍRITO SANTO": "ES","ESPIRITO SANTO": "ES","GOIÁS": "GO","GOIAS": "GO",
    "MARANHÃO": "MA","MARANHAO": "MA","MATO GROSSO": "MT","MATO GROSSO DO SUL": "MS",
    "MINAS GERAIS": "MG","PARÁ": "PA","PARA": "PA","PARAÍBA": "PB","PARAIBA": "PB",
    "PARANÁ": "PR","PARANA": "PR","PERNAMBUCO": "PE","PIAUÍ": "PI","PIAUI": "PI",
    "RIO DE JANEIRO": "RJ","RIO GRANDE DO NORTE": "RN","RIO GRANDE DO SUL": "RS",
    "RONDÔNIA": "RO","RONDONIA": "RO","RORAIMA": "RR","SANTA CATARINA": "SC",
    "SÃO PAULO": "SP","SAO PAULO": "SP","SERGIPE": "SE","TOCANTINS": "TO",
}

mapping_expr = F.create_map([F.lit(x) for pair in uf_map.items() for x in pair])


emendas_filtradas = emendas_filtradas.withColumn("localidade_raw", F.upper(F.col("Localidade_do_gasto__Regionalizacao_").cast("string")))

emendas_filtradas = (
    emendas_filtradas
    # MUNICÍPIO quando tiver " - "
    .withColumn(
        "municipio_extr",
        F.when(
            F.col("localidade_raw").contains(" - "),
            F.trim(F.regexp_extract(F.col("localidade_raw"), r"^(.*?)-", 1))
        )
    )

    # UF encontrada pelo final da string (ex: " - RJ")
    .withColumn(
        "uf_final",
        F.regexp_extract(F.col("localidade_raw"), r"\b([A-Z]{2})$", 1)
    )

    # Nome do estado (para mapear)
    .withColumn(
        "estado_extr",
        F.trim(F.regexp_extract(F.col("localidade_raw"), r"^([A-ZÀ-Ü ]+)", 1))
    )


    # LOCALIDADE SEMPRE UPPERCASE
    .withColumn("localidade_destino", F.col("localidade_raw"))

    # MUNICÍPIO DESTINO
    .withColumn(
        "municipio_destino",
        F.col("municipio_extr")
    )

    # LÓGICA FINAL DA UF
    .withColumn(
        "uf_destino",
        F.when(F.col("municipio_extr").isNotNull(), F.col("uf_final"))               # caso "CIDADE - UF"
         .when(F.col("uf_final") != "", F.col("uf_final"))                          # final = UF
         .otherwise(mapping_expr[F.col("estado_extr")])                              # nome → UF (mapeamento)
    )

    .drop("Localidade_do_gasto__Regionalizacao_","localidade_raw", "municipio_extr", "estado_extr", "uf_final")
)

In [0]:
emendas_filtradas\
    .filter(F.col("uf_destino").isNull())\
    .groupBy("localidade_destino")\
    .count()\
    .orderBy(F.desc("count"))\
    .display()


localidade_destino,count
MÚLTIPLO,7120
NACIONAL,1658
CENTRO-OESTE,28
EXTERIOR,20
NORDESTE,9
SUL,8
SUDESTE,8
NORTE,6


A Lógica está funcionando corretamente, só temos valores vazios quando não temos um estado definido.

#### Função

In [0]:
emendas_filtradas.select("Funcao").printSchema()

root
 |-- Funcao: string (nullable = true)



A coluna de função está como string que é tipo correto da coluna

In [0]:
emendas_filtradas.filter(F.col("Funcao").isNull()).count()

0

Não temos valores nulos para coluna de função

In [0]:
emendas_filtradas.groupBy("Funcao").count().orderBy(F.desc("count")).display()

Funcao,count
Saúde,6788
Encargos especiais,2812
Educação,1411
Assistência social,1258
Defesa nacional,811
Desporto e lazer,741
Direitos da cidadania,673
Cultura,626
Segurança pública,590
Agricultura,570


Todos os 27 tipos de emendas parlamentares são válidos

#### Valores

In [0]:
emendas_filtradas.select("Valor_empenhado", "Valor_liquidado", "Valor_pago").printSchema()

root
 |-- Valor_empenhado: string (nullable = true)
 |-- Valor_liquidado: string (nullable = true)
 |-- Valor_pago: string (nullable = true)



Os valores estão com os formatos incorretos e precisarão ser corrigidos.

In [0]:
emendas_filtradas = emendas_filtradas.withColumn(
    "Valor_empenhado",
    F.regexp_replace(
        F.regexp_replace(F.col("Valor_empenhado"), r"\.", ""),
        ",", "."
    ).cast("double")
)

emendas_filtradas = emendas_filtradas.withColumn(
    "Valor_liquidado",
    F.regexp_replace(
        F.regexp_replace(F.col("Valor_liquidado"), r"\.", ""),
        ",", "."
    ).cast("double")
)

emendas_filtradas = emendas_filtradas.withColumn(
    "Valor_pago",
    F.regexp_replace(
        F.regexp_replace(F.col("Valor_pago"), r"\.", ""),
        ",", "."
    ).cast("double")
)

In [0]:
emendas_filtradas.limit(10).display()

Ano,Funcao,Valor_empenhado,Valor_liquidado,Valor_pago,nome_parlamentar,localidade_destino,municipio_destino,uf_destino
2023,Saúde,6467.0,6467.0,6467.0,TADEU ALENCAR,BODOCÓ - PE,BODOCÓ,PE
2023,Saúde,24570.0,24570.0,24570.0,JUNIOR MANO,TURURU - CE,TURURU,CE
2023,Educação,38997.44,9995.44,9871.44,HUMBERTO COSTA,PERNAMBUCO (UF),null,PE
2023,Comunicações,48876.0,0.0,0.0,PATRUS ANANIAS,MINAS GERAIS (UF),null,MG
2023,Defesa nacional,49944.39,49944.39,49944.39,FREI ANASTACIO RIBEIRO,NACIONAL,null,null
2023,Saúde,50000.0,0.0,0.0,JORGE KAJURU,MÚLTIPLO,null,null
2023,Educação,50000.0,0.0,0.0,PROFESSORA ROSA NEIDE,DISTRITO FEDERAL (UF),null,DF
2023,Saúde,50548.0,50548.0,50548.0,MARCEL VAN HATTEM,VERA CRUZ - RS,VERA CRUZ,RS
2023,Educação,60672.9,23000.0,0.0,HUMBERTO COSTA,PERNAMBUCO (UF),null,PE
2023,Defesa nacional,62920.0,62920.0,62920.0,GENERAL PETERNELLI,NACIONAL,null,null


Para validar as colunas de valores, não podemos ter valores nulos. <Br>
Além disso, o valor empenhado não pode ser menor que o valor liquidado e o valor liquidado não pode ser menor que o valor pago.

In [0]:
colunas_valores = ["Valor_empenhado", "Valor_liquidado", "Valor_pago"]

In [0]:
emendas_filtradas.select([
    F.col(c).isNull().cast("int").alias(c)
    for c in colunas_valores
]).groupBy().sum().show()

+--------------------+--------------------+---------------+
|sum(Valor_empenhado)|sum(Valor_liquidado)|sum(Valor_pago)|
+--------------------+--------------------+---------------+
|                   0|                   0|              0|
+--------------------+--------------------+---------------+



Não temos valores nulos para as colunas de valores

In [0]:

emendas_filtradas.select(colunas_valores).describe().display()

summary,Valor_empenhado,Valor_liquidado,Valor_pago
count,17693,17693,17693
mean,3826757.5878121383,3192164.2684790795,3172365.670843279
stddev,5604895.361145977,5516565.830328025,5513678.438611269
min,301.48,0.0,0.0
max,6.8539715E7,6.8539715E7,6.8539715E7


Toda emenda contém ao menos um valor empenhado

In [0]:
emendas_filtradas.select(F.max("Valor_empenhado")).show()

+--------------------+
|max(Valor_empenhado)|
+--------------------+
|         6.8539715E7|
+--------------------+



In [0]:
emendas_filtradas.orderBy(F.desc("valor_empenhado")).limit(10).display()

Ano,Funcao,Valor_empenhado,Valor_liquidado,Valor_pago,nome_parlamentar,localidade_destino,municipio_destino,uf_destino
2025,Saúde,6.8539715E7,6.8539715E7,6.8539715E7,EDUARDO BRAGA,MÚLTIPLO,null,null
2025,Saúde,6.8539714E7,6.8539714E7,6.8485356E7,JORGE KAJURU,MÚLTIPLO,null,null
2024,Saúde,6.713485E7,6.713485E7,6.713485E7,EDUARDO BRAGA,MÚLTIPLO,null,null
2025,Saúde,6.5539711E7,6.5539711E7,6.5539711E7,ROMARIO,MÚLTIPLO,null,null
2024,Saúde,6.013485E7,0.0,0.0,IRAJA,MÚLTIPLO,null,null
2023,Saúde,4.3321132E7,4.3321132E7,4.3321132E7,JOSE SERRA,MÚLTIPLO,null,null
2025,Saúde,4.26E7,3.656E7,4.015E7,MARA GABRILLI,MÚLTIPLO,null,null
2024,Saúde,3.716735E7,3.713485E7,3.713485E7,AUGUSTA BRITO,CEARÁ (UF),null,CE
2024,Saúde,3.6468246E7,2.9439201E7,2.9439201E7,ROGERIO CARVALHO,MÚLTIPLO,null,null
2024,Saúde,3.54E7,3.48E7,3.48E7,ANGELO CORONEL,MÚLTIPLO,null,null


Temos emendas parlamentares de 68 milhões de reais com esses valores pagos

In [0]:
df_invalidos = emendas_filtradas.filter(
    (F.col("Valor_empenhado") < F.col("Valor_liquidado")) |
    (F.col("Valor_liquidado") < F.col("Valor_pago"))
)

In [0]:
df_invalidos.display()

Ano,Funcao,Valor_empenhado,Valor_liquidado,Valor_pago,nome_parlamentar,localidade_destino,municipio_destino,uf_destino
2024,Transporte,100000.0,82670.49,100000.0,CAPITAO ALBERTO NETO,NACIONAL,null,null
2024,Transporte,200000.0,184024.67,200000.0,LUIS CARLOS HEINZE,NACIONAL,null,null
2024,Urbanismo,1499939.79,95041.21,1331009.11,ANDRE FIGUEIREDO,CEARÁ (UF),null,CE
2025,Cultura,100000.0,100000.0,200000.0,JUNIO AMARAL,MINAS GERAIS (UF),null,MG
2025,Relações exteriores,147936.96,150796.92,150796.92,GILBERTO ABRAMO,EXTERIOR,null,null
2025,Defesa nacional,161656.28,20821.0,26260.05,LUCIANO BIVAR,NACIONAL,null,null
2025,Assistência social,199995.8,199995.8,376937.9,CHICO RODRIGUES,NACIONAL,null,null
2025,Defesa nacional,199999.31,199999.31,249998.62,LAURA CARNEIRO,NACIONAL,null,null
2025,Agricultura,200000.0,208341.61,171587.94,AUGUSTO COUTINHO,PERNAMBUCO (UF),null,PE
2025,Assistência social,200000.0,400000.0,200000.0,LUIZIANNE LINS,NACIONAL,null,null


Podemos notar 96 emendas que não respeitam a regra de o valor ser empenhado (Dedicado a um tema), liquidado (Designado a pagamento) e pago (Quando é efetivamente realizado a transfêrencia de dinheiro).

In [0]:
df_invalidos_2 = emendas_filtradas.filter(
    (F.col("Valor_empenhado") < F.col("Valor_liquidado")) 
)

In [0]:
df_invalidos_2.display()

Ano,Funcao,Valor_empenhado,Valor_liquidado,Valor_pago,nome_parlamentar,localidade_destino,municipio_destino,uf_destino
2025,Relações exteriores,147936.96,150796.92,150796.92,GILBERTO ABRAMO,EXTERIOR,null,null
2025,Agricultura,200000.0,208341.61,171587.94,AUGUSTO COUTINHO,PERNAMBUCO (UF),null,PE
2025,Assistência social,200000.0,400000.0,200000.0,LUIZIANNE LINS,NACIONAL,null,null
2025,Administração,200000.0,211890.79,144189.39,KIKO CELEGUIM,SÃO PAULO (UF),null,SP
2025,Educação,206261.84,211096.84,182193.84,PEDRO CAMPOS,PERNAMBUCO (UF),null,PE
2025,Educação,295894.0,379694.0,111624.6,ROGERIO MARINHO,RIO GRANDE DO NORTE (UF),null,RN
2025,Educação,300000.0,304600.0,300000.0,PEDRO CAMPOS,PERNAMBUCO (UF),null,PE
2025,Assistência social,345065.52,730764.09,0.0,LUCIANO BIVAR,PERNAMBUCO (UF),null,PE
2025,Encargos especiais,400000.0,800000.0,400000.0,ANTONIO BRITO,BAHIA (UF),null,BA
2025,Assistência social,499930.64,699928.78,282645.75,VICENTINHO,SÃO PAULO (UF),null,SP


Temos 55 emendas com valores empenhados menores do que os valores liquidados. Ou seja, foram gastos mais do que planejado.

### Salvar tabela

In [0]:
# Renomear colunas e agregar emendas com as mesmas informações
emendas_silver = (
    emendas_filtradas
    .select(
        F.col("nome_parlamentar"),
        F.col("Ano").alias("ano"),
        F.col("Funcao").alias("funcao"),
        F.col("Valor_empenhado").alias("valor_empenhado"),
        F.col("Valor_liquidado").alias("valor_liquidado"),
        F.col("Valor_pago").alias("valor_pago"),
        F.col("localidade_destino").alias("localidade_destino"),
        F.col("municipio_destino").alias("municipio_destino"),
        F.col("uf_destino").alias("uf_destino")
    )
    .groupBy(
        "nome_parlamentar",
        "ano",
        "funcao",
        "localidade_destino",
        "municipio_destino",
        "uf_destino"
    )
    .agg(
        F.sum("valor_empenhado").alias("valor_empenhado"),
        F.sum("valor_liquidado").alias("valor_liquidado"),
        F.sum("valor_pago").alias("valor_pago")
    )
)

In [0]:
emendas_silver.limit(5).display()

nome_parlamentar,ano,funcao,localidade_destino,municipio_destino,uf_destino,valor_empenhado,valor_liquidado,valor_pago
TADEU ALENCAR,2023,Saúde,BODOCÓ - PE,BODOCÓ,PE,6467.0,6467.0,6467.0
JUNIOR MANO,2023,Saúde,TURURU - CE,TURURU,CE,24570.0,24570.0,24570.0
HUMBERTO COSTA,2023,Educação,PERNAMBUCO (UF),null,PE,299670.33999999997,89907.44,9871.44
PATRUS ANANIAS,2023,Comunicações,MINAS GERAIS (UF),null,MG,48876.0,0.0,0.0
FREI ANASTACIO RIBEIRO,2023,Defesa nacional,NACIONAL,null,null,49944.39,49944.39,49944.39


In [0]:
emendas_silver.count()

12374

Temos agora 12374 emendas parlamentares após a remoção de emendas de bancadas, comissões e agregação de emendas com mesmos metadados

In [0]:
emendas_silver.write.format("delta").mode("overwrite").saveAsTable("mvp.silver.emendas")

In [0]:
spark.sql("""
COMMENT ON TABLE mvp.silver.emendas IS 'Tabela agregada de emendas parlamentares, contendo valores empenhados, liquidados e pagos por parlamentar, ano, função e destino.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.nome_parlamentar IS 'Nome padronizado do parlamentar autor da emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.ano IS 'Ano de referência da emenda parlamentar. (Valores válidos: 2023, 2024, 2025)'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.funcao IS 'Função orçamentária associada à emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.localidade_destino IS 'Localidade de destino dos recursos da emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.municipio_destino IS 'Município de destino dos recursos da emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.uf_destino IS 'Unidade Federativa (UF) de destino dos recursos da emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.valor_empenhado IS 'Valor total empenhado na emenda parlamentar. (Valores maiores que 0)'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.valor_liquidado IS 'Valor total liquidado na emenda parlamentar. (Valores maiores ou iguais a 0)'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.valor_pago IS 'Valor total pago na emenda parlamentar.(Valores maiores ou iguais a 0)'
""")

DataFrame[]

## Tabela de parlamentares

In [0]:
# Carregar a tabela de parlamentares
parlamentares = spark.read.table("mvp.bronze.parlamentares")

In [0]:
# Visualizar as primeiras 5 linhas
parlamentares.limit(5).display()

email,id,idLegislatura,nome,siglaPartido,siglaUf,uri,uriPartido,urlFoto
dep.acaciofavacho@camara.leg.br,204379,57,Acácio Favacho,MDB,AP,https://dadosabertos.camara.leg.br/api/v2/deputados/204379,https://dadosabertos.camara.leg.br/api/v2/partidos/36899,https://www.camara.leg.br/internet/deputado/bandep/204379.jpg
dep.adailfilho@camara.leg.br,220714,57,Adail Filho,REPUBLICANOS,AM,https://dadosabertos.camara.leg.br/api/v2/deputados/220714,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,https://www.camara.leg.br/internet/deputado/bandep/220714.jpg
dep.adolfoviana@camara.leg.br,204560,57,Adolfo Viana,PSDB,BA,https://dadosabertos.camara.leg.br/api/v2/deputados/204560,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,https://www.camara.leg.br/internet/deputado/bandep/204560.jpg
dep.adrianaventura@camara.leg.br,204528,57,Adriana Ventura,NOVO,SP,https://dadosabertos.camara.leg.br/api/v2/deputados/204528,https://dadosabertos.camara.leg.br/api/v2/partidos/37901,https://www.camara.leg.br/internet/deputado/bandep/204528.jpg
dep.adrianodobaldy@camara.leg.br,121948,57,Adriano do Baldy,PP,GO,https://dadosabertos.camara.leg.br/api/v2/deputados/121948,https://dadosabertos.camara.leg.br/api/v2/partidos/37903,https://www.camara.leg.br/internet/deputado/bandep/121948.jpg


In [0]:
# Colunas da tabela
parlamentares.columns

['email',
 'id',
 'idLegislatura',
 'nome',
 'siglaPartido',
 'siglaUf',
 'uri',
 'uriPartido',
 'urlFoto']

In [0]:
# Selecionar apenas as colunas desejadas
colunas_relevantes_parlamentares = ['nome',
 'siglaPartido',
 'siglaUf'
]
parlamentares_filtrados = parlamentares.select(colunas_relevantes_parlamentares)

parlamentares_filtrados.limit(5).display()

nome,siglaPartido,siglaUf
Acácio Favacho,MDB,AP
Adail Filho,REPUBLICANOS,AM
Adolfo Viana,PSDB,BA
Adriana Ventura,NOVO,SP
Adriano do Baldy,PP,GO


### Análise de duplicidade

In [0]:
chave = [
    "nome",
    "siglaPartido",
    "siglaUF"
]

duplicados = (
    parlamentares_filtrados
    .groupBy(chave)
    .count()
    .filter(F.col("count") > 1)
)

duplicados.count()

0

Não temos parlamentares duplicados

### Análise de nulos

In [0]:
# Contagem de nulos para cada coluna
nulos = parlamentares_filtrados.select([
    F.sum(F.col(c).isNull().cast("int")).alias(c) for c in colunas_relevantes_parlamentares
])

nulos.show()

+----+------------+-------+
|nome|siglaPartido|siglaUf|
+----+------------+-------+
|   0|           0|      0|
+----+------------+-------+



Não temos nulos na colunas da tabela de parlamentares

### Análise das colunas

#### Nome do Parlamentar

Para as colunas de nome precisamos realizar uma padronização. Por isso, precisamos remover acentos e utilizar o upper case

In [0]:
def padroniza_nome(coluna):
    return F.upper(
        F.translate(
            F.col(coluna),
            "áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ",
            "aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC"
        )
    )

In [0]:
parlamentares_filtrados = parlamentares_filtrados.withColumn("nome_parlamentar", padroniza_nome("nome"))

In [0]:
parlamentares_filtrados.groupBy("nome_parlamentar").count().orderBy(F.desc("count")).limit(5).display()

nome_parlamentar,count
ADRIANA VENTURA,1
ADAIL FILHO,1
ACACIO FAVACHO,1
ADOLFO VIANA,1
ADRIANO DO BALDY,1


#### Sigla do Partido

In [0]:
parlamentares_filtrados.groupBy("siglaPartido").count().orderBy(F.desc("count")).display()

siglaPartido,count
PL,86
PT,67
UNIÃO,59
PP,50
PSD,47
REPUBLICANOS,45
MDB,42
PODE,17
PDT,16
PSB,16


Podemos ver 20 partidos dentro da base de parlamentares e todos são válidos

#### Sigla da UF

In [0]:
parlamentares_filtrados.groupBy("siglaUf").count().orderBy(F.desc("count")).display()

siglaUf,count
SP,70
MG,53
RJ,45
BA,39
RS,31
PR,30
PE,25
CE,22
MA,18
PA,17


Podemos ver as 27 unidades federativas e todas com ao menos 8 deputados federais

### Salvar tabela

In [0]:
parlamentares_filtrados.limit(5).display()

nome,siglaPartido,siglaUf,nome_parlamentar
Acácio Favacho,MDB,AP,ACACIO FAVACHO
Adail Filho,REPUBLICANOS,AM,ADAIL FILHO
Adolfo Viana,PSDB,BA,ADOLFO VIANA
Adriana Ventura,NOVO,SP,ADRIANA VENTURA
Adriano do Baldy,PP,GO,ADRIANO DO BALDY


In [0]:
# Renomear colunas e agregar emendas com as mesmas informações
parlamentares_silver = parlamentares_filtrados.select("nome_parlamentar", "siglaPartido", "siglaUf")

In [0]:
parlamentares_silver.limit(5).display()

nome_parlamentar,siglaPartido,siglaUf
ACACIO FAVACHO,MDB,AP
ADAIL FILHO,REPUBLICANOS,AM
ADOLFO VIANA,PSDB,BA
ADRIANA VENTURA,NOVO,SP
ADRIANO DO BALDY,PP,GO


In [0]:
parlamentares_silver.count()

512

In [0]:
parlamentares_silver.write.format("delta").mode("overwrite").saveAsTable("mvp.silver.parlamentares")

In [0]:
spark.sql("""
  COMMENT ON TABLE mvp.silver.parlamentares IS 'Tabela silver contendo parlamentares, com nome, partido e UF';
""")

spark.sql("""
  COMMENT ON COLUMN mvp.silver.parlamentares.nome_parlamentar IS 'Nome do parlamentar';
""")

spark.sql("""
  COMMENT ON COLUMN mvp.silver.parlamentares.siglaPartido IS 'Sigla do partido do parlamentar';
""")

spark.sql("""
  COMMENT ON COLUMN mvp.silver.parlamentares.siglaUf IS 'Sigla da unidade federativa do parlamentar';
""")

DataFrame[]